In [1]:

import os,time
os.environ['CUDA_VISIBLE_DEVICES']='2'
import copy
import logging
import numpy as np
import pandas as pd
from datetime import datetime,timedelta,date
import time
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel, get_linear_schedule_with_warmup, \
    default_data_collator, TrainingArguments, Trainer
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
from scipy.stats import pearsonr, spearmanr
from transformers import AutoTokenizer
from transformers import BertConfig, BertTokenizer,BertTokenizerFast
from datasets import Dataset
from typing import List,Union
import tqdm
import re
import json

os.chdir('/home/stops/Work_space/NLP_work/Med_assit_chatglm')


from db_config_taiyi import DB ## load data from pgsql

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s - %(message)s')
logger.info('Starting')



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/stops/miniconda3/envs/LLM_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/stops/miniconda3/envs/LLM_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA SETUP: Loading binary /home/stops/miniconda3/envs/LLM_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/stops/miniconda3/envs/LLM_env/lib/python3.8/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
2024-01-02 17:14:25,524 - INFO : Starting


In [2]:
##################################
## Instruct data
##################################

data_file='/home/stops/Work_space/NLP_models/train_2M_CN/train_2M_CN.json'

belle_instruct_data=[]
with open(data_file,'r') as f :
    for sub_data in f.readlines()[:50]:
        sub_data_dict=json.loads(sub_data)
        belle_instruct_data.append(sub_data_dict)
logger.info(f'belle_instruct_data nums: {len(belle_instruct_data)}')
logger.info(f'examples: {belle_instruct_data[:2]}')

2024-01-02 17:14:32,357 - INFO : belle_instruct_data nums: 50
2024-01-02 17:14:32,358 - INFO : examples: [{'instruction': '对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  print(add(2,3))', 'input': '', 'output': '这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是说函数在 `return` 语句处结束，因此 `print` 语句永远不会被执行。\n要解决这个问题，只需在 `return` 语句之前移到 `print` 语句。修改后的代码如下所示：\n```\ndef add(a, b):\n    print(a + b)\n    return a + b\nadd(2, 3)\n```\n这样就可以在调用 `add` 函数时打印出结果，并在函数内部返回 `a + b` 值，使其在需要时可以接收和使用。同时，注意在调用 `add` 函数时，不需要再使用 `print` 方法了。'}, {'instruction': '为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。\n', 'input': '', 'output': '为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜太长；\n2. 名称应该与应用程序的主要功能或特点相关；\n3. 名称应该引起人们的兴趣和好奇心。\n举个例子，如果我们的应用程序是一款健康管理应用程序，那么可以考虑以下几个名称：\n1. HealthBoss\n2. FitMeUp\n3. HealthMate\n4. BodyBest\n5. HealthLab\n其中，HealthBoss强调应用程序可帮助用户成为健康的老板，FitMeUp强调应用程序可帮助用户达到健康生活的目标，HealthMate强调应用程序是用户健康的伙伴，BodyBest强调应用程序可通过健康管理帮助用户获得最佳身体状态，HealthLab强调应用程序可帮助用户进行健康诊断和实验。\n一句简短的介绍可包含以下几个要素：\n1. 应用程序的名称；\n2. 应用程序的主要功

In [3]:
belle_instruct_df=pd.DataFrame.from_dict(belle_instruct_data)
belle_instruct_df=belle_instruct_df.applymap(lambda x:x.strip())
print(belle_instruct_df.shape)
print(belle_instruct_df.head())
print(belle_instruct_df.isnull().sum())

(50, 3)
                                         instruction input  \
0  对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  p...         
1                     为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。         
2  在给定的代码示例中，修复一个特定的错误。\n以下是一个Python代码示例：\ndef su...         
3  根据文本生成一段简短的摘要。\n文本：据报道，乔治·华盛顿于1732年2月22日出生在维珍尼...         
4                                  回答下面这个问题：什么是人工智能？         

                                              output  
0  这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是...  
1  为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜...  
2  该Python代码中的错误是调用sum()函数时传递了3个参数，而该函数只接受2个参数。因此...  
3                     乔治·华盛顿出生在1732年2月22日，位于维吉尼亚殖民地。  
4  人工智能（Artificial Intelligence，简称 AI）是一种旨在模拟、复制、...  
instruction    0
input          0
output         0
dtype: int64


In [4]:
belle_instruct_df.loc[:,'instruct_len']=belle_instruct_df.apply(lambda x:len(x['instruction']),axis=1)
print(belle_instruct_df['instruct_len'].describe(percentiles=[0.01,0.1,0.2,0.25,0.5,0.75,0.8,0.9,0.9999]))

count      50.000000
mean       47.440000
std        24.964595
min        11.000000
1%         13.450000
10%        22.800000
20%        28.400000
25%        30.250000
50%        43.000000
75%        52.000000
80%        61.000000
90%        80.900000
99.99%    131.853000
max       132.000000
Name: instruct_len, dtype: float64


In [5]:
select_instruct_df=belle_instruct_df.loc[belle_instruct_df['instruct_len']<600,['instruction','output']].copy()
print(select_instruct_df.shape)
print(select_instruct_df.head())

(50, 2)
                                         instruction  \
0  对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  p...   
1                     为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。   
2  在给定的代码示例中，修复一个特定的错误。\n以下是一个Python代码示例：\ndef su...   
3  根据文本生成一段简短的摘要。\n文本：据报道，乔治·华盛顿于1732年2月22日出生在维珍尼...   
4                                  回答下面这个问题：什么是人工智能？   

                                              output  
0  这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是...  
1  为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜...  
2  该Python代码中的错误是调用sum()函数时传递了3个参数，而该函数只接受2个参数。因此...  
3                     乔治·华盛顿出生在1732年2月22日，位于维吉尼亚殖民地。  
4  人工智能（Artificial Intelligence，简称 AI）是一种旨在模拟、复制、...  


In [6]:
instruct_prompt_text="""对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。
参考类别 [代码，头脑风暴，健康医疗，闲聊，翻译，文本重写，文本提取摘要，文本扩写，信息抽取，文本理解，文本分类，科学，数学，传记，商业，社会，经济，文化，角色扮演，常识类问答，其他]，同时不要出现没有在上面的类别。

指令文本内容：
"{text}"

任务要求如下：
1.首先给出所属类别的分析过程。
2.再返回结果格式为json，{{"类别":[xxx]}}。
"""



In [7]:
check_user_first_text_result_list=select_instruct_df["instruction"].tolist()

user_first_text_department_result_list=[]
for sub_text in check_user_first_text_result_list:
    sub_u_f_text=instruct_prompt_text.format(text=sub_text)
    user_first_text_department_result_list.append(sub_u_f_text)

logger.info(f"user_first_text_department_result_list nums: {len(user_first_text_department_result_list)}")

2024-01-02 17:14:33,873 - INFO : user_first_text_department_result_list nums: 50


In [8]:
print(user_first_text_department_result_list[:2])



['对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别 [代码，头脑风暴，健康医疗，闲聊，翻译，文本重写，文本提取摘要，文本扩写，信息抽取，文本理解，文本分类，科学，数学，传记，商业，社会，经济，文化，角色扮演，常识类问答，其他]，同时不要出现没有在上面的类别。\n\n指令文本内容：\n"对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  print(add(2,3))"\n\n任务要求如下：\n1.首先给出所属类别的分析过程。\n2.再返回结果格式为json，{"类别":[xxx]}。\n', '对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别 [代码，头脑风暴，健康医疗，闲聊，翻译，文本重写，文本提取摘要，文本扩写，信息抽取，文本理解，文本分类，科学，数学，传记，商业，社会，经济，文化，角色扮演，常识类问答，其他]，同时不要出现没有在上面的类别。\n\n指令文本内容：\n"为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。"\n\n任务要求如下：\n1.首先给出所属类别的分析过程。\n2.再返回结果格式为json，{"类别":[xxx]}。\n']


In [14]:
###################
## load model
###################
model_path="/home/stops/Work_space/NLP_models/Baichuan2-13B-Chat"


from transformers import AutoModel, AutoTokenizer,AutoModelForCausalLM
from transformers.generation.utils import GenerationConfig
from torch.nn.utils.rnn import pad_sequence
from torch.types import Number

def right_padding(sequences: [torch.Tensor], padding_value) -> torch.Tensor:
    return pad_sequence(sequences, batch_first=True, padding_value=padding_value)


def left_padding(sequences: [torch.Tensor], padding_value) -> torch.Tensor:
    return right_padding(
        [seq.flip(0) for seq in sequences],
        padding_value=padding_value,).flip(1)


def init_model():
    #model = AutoModelForCausalLM.from_pretrained(model_path,trust_remote_code=True).to(torch.bfloat16).cuda()
    model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True).half().cuda()
    model.generation_config = GenerationConfig.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path,use_fast=False,trust_remote_code=True)
    logger.info(f"load model path: {model_path}")
    return model, tokenizer


In [15]:
model, tokenizer = init_model()


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2023-12-21 14:54:54,410 - INFO : load model path: /home/stops/Work_space/NLP_models/Baichuan2-13B-Chat


In [16]:
def LLM_batch_infer(batch_text_list:List[str]):
    if isinstance(batch_text_list,str):
        batch_text_list=[batch_text_list]
    user_token_id=195
    assistant_token_id=196
    batch_input_ids=[]
    for sub_text in batch_text_list:
        sub_input_ids = [user_token_id]+tokenizer.encode(text=sub_text)+[assistant_token_id]
        batch_input_ids.append(torch.tensor(sub_input_ids, dtype=torch.long))
    batch_input_len=[len(item) for item in batch_input_ids]
    batch_max_len=max(batch_input_len)
    ## padding-strategy: LEFT
    batch_input_tensor_ids=left_padding(batch_input_ids, padding_value=0)
    test_batch_input_ids=batch_input_tensor_ids.to("cuda")
    batch_outputs = model.generate(test_batch_input_ids)
    batch_response_text_result_list=[]
    for sub_output  in batch_outputs:
        sub_response = tokenizer.decode(sub_output[batch_max_len:], skip_special_tokens=True)
        batch_response_text_result_list.append(sub_response)
    return batch_response_text_result_list


In [25]:
test_text_list=user_first_text_department_result_list[:2]
print("test_text_list: ",test_text_list)

s_time=time.time()
test_infer_texts=LLM_batch_infer(test_text_list)
print("batch cost time : {:.2f}".format(time.time()-s_time))
print("test_infer_texts: ",test_infer_texts)


test_text_list:  ['对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别 [代码，头脑风暴，健康医疗，闲聊，翻译，文本重写，文本提取摘要，文本扩写，信息抽取，文本理解，文本分类，科学，数学，传记，商业，社会，经济，文化，角色扮演，常识类问答，其他]，同时不要出现没有在上面的类别。\n\n指令文本内容：\n"对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  print(add(2,3))"\n\n任务要求如下：\n1.首先给出所属类别的分析过程。\n2.再返回结果格式为json，{"类别":[xxx]}。\n', '对于下面指令文本的意图目的，参考下面已有的类别，给出所属的类别。\n参考类别 [代码，头脑风暴，健康医疗，闲聊，翻译，文本重写，文本提取摘要，文本扩写，信息抽取，文本理解，文本分类，科学，数学，传记，商业，社会，经济，文化，角色扮演，常识类问答，其他]，同时不要出现没有在上面的类别。\n\n指令文本内容：\n"为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。"\n\n任务要求如下：\n1.首先给出所属类别的分析过程。\n2.再返回结果格式为json，{"类别":[xxx]}。\n']
batch cost time : 1.19
test_infer_texts:  ['{\n    "类别": ["代码"]\n}', '{\n    "类别": ["文本扩写", "创意设计"]\n}']


In [26]:
extract_department_text="""[{].*?[}]"""
test_result=re.findall(extract_department_text,test_infer_texts[0])
print("test_result: ",test_result)


test_result:  []


In [27]:
####################
## 预测结果
####################
department_infer_detail_result=[]
department_infer_result=[]
batch_size=2
all_nums=len(user_first_text_department_result_list)
logger.info(f"run nums: {all_nums} , batch_size: {batch_size}")

for idx in range(0,all_nums,batch_size):
    if idx%10==0:
        logger.info("run step: {}, finished: {:.2%}".format(idx,idx/all_nums))
    sub_batch_texts=user_first_text_department_result_list[idx:(idx+batch_size)]
    sub_infer_texts=LLM_batch_infer(sub_batch_texts)
    extract_department_text="""[{].*?[}]"""
    for sub_text in sub_infer_texts:
        sub_result=re.findall(extract_department_text,sub_text)
        department_infer_detail_result.append(sub_text)
        if sub_result:
            sub_result_text=sub_result[0]
        else:
            sub_result_text=None
        department_infer_result.append(sub_result_text)

logger.info(f"department_infer_detail_result nums: {len(department_infer_detail_result)}")
logger.info(f"examples: {department_infer_detail_result[:2]}")


logger.info(f"department_infer_result nums: {len(department_infer_result)}")
logger.info(f"examples: {department_infer_result[:2]}")


2023-12-21 15:13:20,102 - INFO : run nums: 50 , batch_size: 2
2023-12-21 15:13:20,104 - INFO : run step: 0, finished: 0.00%
2023-12-21 15:13:33,598 - INFO : run step: 10, finished: 20.00%
2023-12-21 15:14:16,823 - INFO : run step: 20, finished: 40.00%
2023-12-21 15:14:30,982 - INFO : run step: 30, finished: 60.00%
2023-12-21 15:14:44,899 - INFO : run step: 40, finished: 80.00%
2023-12-21 15:15:06,328 - INFO : department_infer_detail_result nums: 50
2023-12-21 15:15:06,329 - INFO : examples: ['{\n    "类别": ["代码"]\n}', '{\n    "类别": ["文本扩写", "创意设计"]\n}']
2023-12-21 15:15:06,330 - INFO : department_infer_result nums: 50
2023-12-21 15:15:06,331 - INFO : examples: [None, None]


In [28]:
final_department_result_list=[]
for sub_text in department_infer_result:
    #print("sub_text: ",sub_text)
    try:
        sub_dict=eval(sub_text)
        #print("sub_dict: ",sub_dict)
        sub_res=sub_dict["类别"][0]
    except:
        sub_res=None
    final_department_result_list.append(sub_res)

logger.info(f"final_department_result_list nums: {len(final_department_result_list)}")
logger.info(f"examples: {final_department_result_list[:2]}")


2023-12-21 15:15:06,337 - INFO : final_department_result_list nums: 50
2023-12-21 15:15:06,338 - INFO : examples: [None, None]


In [29]:


save_df=select_instruct_df.head(len(final_department_result_list)).copy()

save_df.loc[:,"detail_result"]=department_infer_detail_result
save_df.loc[:,"dict_result"]=department_infer_result
save_df.loc[:,"result"]=final_department_result_list

print(save_df.shape)
print(save_df.head())
print(save_df.isnull().sum())

save_mode=True
save_file="output_data/Instruct_prediction_0918_df.xlsx"
if save_mode:
    save_df.to_excel(save_file,index=False)
    logger.info(f"data nums: {save_df.shape}, save file: {save_file}")



2023-12-21 15:15:06,373 - INFO : data nums: (50, 5), save file: output_data/Instruct_prediction_0918_df.xlsx


(50, 5)
                                         instruction  \
0  对以下代码进行调试。\ndef add(a,b):\n  return a + b\n  p...   
1                     为一个新的智能手机应用程序起个优秀的名称并写一句简短的介绍。   
2  在给定的代码示例中，修复一个特定的错误。\n以下是一个Python代码示例：\ndef su...   
3  根据文本生成一段简短的摘要。\n文本：据报道，乔治·华盛顿于1732年2月22日出生在维珍尼...   
4                                  回答下面这个问题：什么是人工智能？   

                                              output  \
0  这段代码的问题是在函数内部， `print` 语句出现在 `return` 语句之后，也就是...   
1  为一个新的智能手机应用程序起一个优秀的名称可以考虑以下几点：\n1. 名称应该简单易记，不宜...   
2  该Python代码中的错误是调用sum()函数时传递了3个参数，而该函数只接受2个参数。因此...   
3                     乔治·华盛顿出生在1732年2月22日，位于维吉尼亚殖民地。   
4  人工智能（Artificial Intelligence，简称 AI）是一种旨在模拟、复制、...   

                                       detail_result        dict_result  \
0                             {\n    "类别": ["代码"]\n}               None   
1                   {\n    "类别": ["文本扩写", "创意设计"]\n}               None   
2                             {\n    "类别": ["代码"]\n}               None   
3  1. 所属类别的分析过程：这段